In [163]:
import numpy as np
import random

from util import read_nxgraph
from util import obj_maxcut

In [164]:
class Bucket:
    def __init__(self, gmin, gmax, solution, graph):
        self.gmin = gmin
        self.gmax = gmax
        self.solution = solution
        self.graph = graph
        self.numBuckets = gmax - gmin + 1

        self.bucketsA = [[] for _ in range(self.numBuckets)]
        self.bucketsB = [[] for _ in range(self.numBuckets)]

        self.maxgainA = None
        self.maxgainB = None

        self.vertexMap = {}

    def calcGain(self, vertex):
        newGraph = self.solution
        vertexVal = self.solution[vertex]
        if vertexVal == 0:
            newGraph[vertex] = 1
        else:
            newGraph[vertex] = 0

        return obj_maxcut(newGraph, self.graph) - obj_maxcut(self.solution, self.graph)

    def addVertex(self, vertex, partition):
        """Add a vertex with a given gain to a specific partition."""
        gain = self.calcGain(vertex)
        bucket_array = self.bucketsA if partition == 1 else self.bucketsB

        bucket_array[vertex].append(vertex)
        self.vertexMap[vertex] = (partition, gain, vertex)
        self.update_maxgain(partition, vertex)

    def removeVertex(self, vertex):
        """Remove a vertex from its current bucket."""
        if vertex in self.vertexMap:
            partition, _, index = self.vertexMap[vertex]
            bucket_array = self.bucketsA if partition == 1 else self.bucketsB

            bucket_array[index].remove(vertex)
            del self.vertexMap[vertex]
            self.update_maxgain(partition)

    def update_gain(self, vertex, new_gain):
        """Update the gain of a vertex."""
        if vertex in self.vertexMap:
            partition, _, _ = self.vertexMap[vertex]
            self.removeVertex(vertex)
            self.addVertex(vertex, new_gain, partition)

    def move_vertex(self, vertex):
        """Move a vertex from one partition to the other."""
        if vertex in self.vertexMap:
            partition, gain, _ = self.vertexMap[vertex]
            self.removeVertex(vertex)
            new_partition = 2 if partition == 1 else 1
            self.addVertex(vertex, gain, new_partition)

    def update_maxgain(self, partition, start_index=None):
        """Update the maxgain pointer for the specified partition."""
        bucket_array = self.bucketsA if partition == 1 else self.bucketsB
        maxgain_attr = 'maxgainA' if partition == 1 else 'maxgainB'

        if start_index is None:
            for i in range(self.numBuckets - 1, -1, -1):
                if bucket_array[i]:
                    setattr(self, maxgain_attr, i)
                    return
            setattr(self, maxgain_attr, None)
        elif bucket_array[start_index]:
            current_max = getattr(self, maxgain_attr)
            setattr(self, maxgain_attr, max(current_max or 0, start_index))
        else:
            self.update_maxgain(partition)

    def get_best_move(self):
        bestA = None if self.maxgainA is None else self.bucketsA[self.maxgainA][-1]
        bestB = None if self.maxgainB is None else self.bucketsB[self.maxgainB][-1]

        return bestA if self.maxgainA >= self.maxgainB else bestB


In [165]:
graph = read_nxgraph('.././data/gset/gset_14.txt')
currSolution = list(np.random.randint(0, 2, graph.number_of_nodes()))
bucket = Bucket(graph.number_of_nodes() * -1, graph.number_of_nodes(), currSolution, graph)
currVal = obj_maxcut(currSolution, graph)
bestSolution = currSolution
bestVal = currVal
prevSolution = currSolution
omega = 0
iteration = 0
L0 = 0.01*graph.number_of_nodes()
T = 1000
gamma = random.randint(3,(graph.number_of_nodes()/10))
P0 = 0.8
Q = 0.5

In [166]:
def perturbation(C, L, H, iteration, omega):
    if omega == 0:
        C = perturb(C,L,B)
    else:
        if (np.exp(-(omega)/T) > P0):
            P = np.exp(-(omega)/T)
        else:
            P = P0

        randProb = random.random()

        if (randProb < (P * Q)):
            perturb(C,L,A1)
        elif (randProb < (P * Q + P (1-Q))):
            perturb(C,L,A2)
        else:
            perturb(C,L,B)

    return C

In [167]:
def perturb(C,L,M):
    for i in range(L):
        m = M
        #currSolution = currSolution xor M
        H = iteration + gamma
        iteration = iteration + 1
    
    return C

In [168]:
for i in range(graph.number_of_nodes()):
    if(currSolution[i] == 0):
        bucket.addVertex(i, 0)
    else:
        bucket.addVertex(i, 1)

In [169]:
print(bucket.get_best_move())

799


In [170]:
while(iteration < 200000*graph.number_of_nodes()):
    #combo = [a ^ b for a,b in zip(currSolution, m)]
    combo = None
    while(obj_maxcut(combo, graph) > obj_maxcut(currSolution, graph)):
        currVal = obj_maxcut(combo)
        currSolution = combo
        H = iteration + gamma 
        iteration = iteration + 1
    
    if currVal > bestVal:
        bestSolution = currSolution
        bestVal = currVal
        omega = 0
    else:
        omega = omega + 1

    # Determine the number of perturbation moves L to be applied to C

    if omega > T:
        # Search is stagnating, random perturbation is required
        omega = 0
    
    if currSolution == prevSolution:
        L = L + 1
    else:
        L = L0
    
    prevSolution = currSolution
    currSolution = perturbation(currSolution, L, H, iteration, omega)

TypeError: object of type 'NoneType' has no len()